In [1]:
#Imports/loads Model
import spacy
from spacy import displacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

C:\Users\alexd\AppData\Local\Temp\ipykernel_3372\18877623.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
'''
# Open the input file for reading
doc_file_path = "test_text.txt"
with open(doc_file_path, 'r') as input_file:
    # Read the content of the file
    content = input_file.read() 
'''
content = "Drone has a camera, 3 batteries, a gyroscope, a sticker, and 4 propellors and must fly. John, walks his dog at the park. We are flying to Seattle with our three dogs."

In [13]:
#Initialize Vars
document = nlp(content) 

In [14]:
#follows down NL list of attibutes
def conjTrail(token, lst):
    for child in token.children:
        if child.dep_ == "conj":
            lst.append(child)
            return conjTrail(child, lst)
    return lst

In [15]:
#follows preposition to pobj
def prepToPobj(child):
    lst = []
    #checks for prepositional modifier connecting attribute to subject
    if child.dep_ == "prep":
        #checks for an object of preposition in the children of the prepositional modifier
        for childI in child.children:
            if childI.dep_ == "pobj":
                lst.append(childI)
    return lst

In [16]:
#NL = Natural Language
#Spaghetti code from trial and error testing
sentanceCount = 0
for sentence in document.sents:
    subjectCount = 0
    for token in sentence:
        #find subject    
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass" and token.lemma_ != "it":
            attributes = []
            actions = []
            subject = token
            subjectCount += 1
            #look for action verbs connected to the subject
            for parent in token.ancestors:
                if parent.dep_ == "ROOT" or parent.pos_ == "AUX" and parent not in actions:
                    actions.append(parent)
                    for child in parent.children:
                        #checks for actions via conjuction dep
                        if child.dep_ == "conj" and child not in actions:
                            actions.append(child)
                        #checks for direct object
                        if child.dep_ == "dobj": 
                            attributes.append(child)
                            for childI in child.children:
                                #checks for token with appositional modifier connected to the direct object
                                if childI.dep_ == "appos":
                                    attributes.append(childI)
                                    #looks for chain of conjunctions, if found recursive method called to handle rest of NL list
                                    if any(childJ.dep_ == "conj" for childJ in childI.children):
                                        attributes = attributes + conjTrail(childI,[])
                        #checks for prepositional modifier connecting attribute to subject
                        attributes = attributes + prepToPobj(child)
                        #checks for adjectival complement connecting adjective to subject
                        if child.dep_ == "acomp":
                            for childI in child.children:
                                #checks for prepositional modifier connecting attribute to subject
                                attributes = attributes + prepToPobj(childI)
                        #checks for open clausal complement
                        if child.dep_ == "xcomp":
                            for childI in child.children:
                                if childI.dep_ == "dobj":
                                    attributes.append(childI)
                                    for childJ in childI.children:
                                        #checks for prepositional modifier connecting attribute to subject
                                        attributes = attributes + prepToPobj(childJ)
            #print/output  <sentance #> : <subject #> --> [subject] - [actions] - [attributes]
            print("Sentance#",sentanceCount," Subject#",subjectCount,"-->","[", subject, "] -", actions, "-", attributes)
    sentanceCount += 1

Sentance# 0  Subject# 1 --> [ Drone ] - [has, fly] - [camera, batteries, cat, dog, propellors]
Sentance# 1  Subject# 1 --> [ John ] - [walks] - [dog]
Sentance# 2  Subject# 1 --> [ We ] - [flying] - [Seattle, dogs]
